In [2]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

# Load data
stats_df = pd.read_csv("CleanedData/player_stats_cleaned.csv")
radar_df = pd.read_csv("CleanedData/player_radar.csv")

# Merge the necessary parts of datasets
df = pd.merge(stats_df[['player', 'goals', 'aerials_won']], radar_df[['player', 'passes_percentage', 'shots_percentage', 'defensive']], on='player')

# Calculate tournament averages
average_data = df[['goals', 'aerials_won', 'passes_percentage', 'shots_percentage', 'defensive']].mean().to_dict()

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='player-one-dropdown',
                options=[{'label': player, 'value': player} for player in df['player'].unique()],
                value=None,
                placeholder="Select the first player"
            ),
            dcc.Dropdown(
                id='player-two-dropdown',
                options=[{'label': player, 'value': player} for player in df['player'].unique()],
                value=None,
                placeholder="Select the second player"
            )
        ], width=3),
        dbc.Col(dcc.Graph(id='radar-chart'), width=6),
        dbc.Col(html.Div(id='metric-colors'), width=3)
    ], justify="center")
])

@app.callback(
    Output('radar-chart', 'figure'),
    [Input('player-one-dropdown', 'value'),
     Input('player-two-dropdown', 'value')]
)
def update_radar_chart(player_one, player_two):
    categories = ['Goals', 'Aerials Won', 'Passes Percentage', 'Shots Percentage', 'Defensive']

    fig = go.Figure()

    if player_one:
        data_one = df[df['player'] == player_one].iloc[0]
        fig.add_trace(go.Scatterpolar(
            r=[data_one['goals'], data_one['aerials_won'], data_one['passes_percentage'], data_one['shots_percentage'], data_one['defensive']],
            theta=categories,
            fill='toself',
            name=player_one
        ))

    if player_two:
        data_two = df[df['player'] == player_two].iloc[0]
        fig.add_trace(go.Scatterpolar(
            r=[data_two['goals'], data_two['aerials_won'], data_two['passes_percentage'], data_two['shots_percentage'], data_two['defensive']],
            theta=categories,
            fill='toself',
            name=player_two
        ))

    # Add tournament average
    fig.add_trace(go.Scatterpolar(
        r=[average_data['goals'], average_data['aerials_won'], average_data['passes_percentage'], average_data['shots_percentage'], average_data['defensive']],
        theta=categories,
        fill='toself',
        name='Tournament Average'
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )),
        showlegend=True
    )

    return fig

@app.callback(
    Output('metric-colors', 'children'),
    [Input('radar-chart', 'figure')]
)
def update_metric_colors(figure):
    # Generate color information based on the figure
    return html.Ul([html.Li(f"{trace['name']} - {trace['line']['color']}", style={'color': trace['line']['color']}) for trace in figure['data']])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
